In [7]:
from neo4j import GraphDatabase
import pandas as pd

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "passwd123"))

In [8]:
df = pd.read_csv('skills_with_tags.csv')

FIELDS = df.columns[1:]
FIELDS

Index(['software development', 'web development', 'mobile development',
       'cloud computing', 'devops', 'networking', 'cybersecurity',
       'data science', 'machine learning', 'data engineering',
       'system administration', 'quality assurance', 'project management',
       'ui design', 'systems architecture', 'game development', 'soft skills',
       'certifications', 'compliance'],
      dtype='object')

In [9]:
def create_node(tx, skill, **kwargs):
    query = "CREATE (s:Skill {name: $skill})"
    tx.run(query, skill=skill)

def create_category(tx, category, **kwargs):
    query = "CREATE (c:Category {name: $category})"
    tx.run(query, category=category)

def initialize_category(tx):
    for field in FIELDS:
        create_category(tx, field)
    print('Categories initialized.')

def merge_relationships(tx, skill, category, relevance_level, **kwargs):
    if relevance_level == 0:
        return
    query = """
    MATCH (s:Skill {name: $skill}), (c:Category {name: $category})
    MERGE (s)-[r:RELEVANT_TO]->(c)
    ON CREATE SET r.relevance = $relevance_level
    """
    tx.run(query, skill=skill, category=category, relevance_level=int(relevance_level))

In [10]:
with driver.session() as session:
    session.execute_write(initialize_category)

Categories initialized.


In [ ]:
import csv

# Open CSV and create skill nodes and relationships
with open('skills_with_tags.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        skill_name = row["skill name"]
        
        # Create skill node
        session.execute_write(create_node, skill_name)
        
        # For each category, create a relationship if relevant
        for category in FIELDS:
            relevance_level = row[category]
            session.execute_write(merge_relationships, skill_name, category, relevance_level)


In [11]:
driver.close()